In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import os

os.chdir('..') # Change to parent directory
cwd = os.getcwd() # Get current working directory
print(cwd)

C:\Users\templ\OneDrive\Desktop\AI\Hackathon\Amazon


In [3]:
# Read the dataset using Dask
dataset = pd.read_csv('dataset/universal_train.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
# dataset =  dataset.sample(n=20000)

In [4]:
dataset.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH', 'PRODUCT_LENGTH_NONORM'],
      dtype='object')

In [5]:
dataset = dataset.drop(columns = ['Unnamed: 0'],axis = 1)

In [6]:
dataset

,0,1,2,3,4,5,6,7,8,9,10,PRODUCT_TYPE_ID,PRODUCT_LENGTH,PRODUCT_LENGTH_NONORM
0,0.033740,0.053593,0.119451,-0.002144,-0.023888,-0.009354,-0.013213,-0.011067,-0.052480,-0.005460,-0.048928,-0.592630,-0.001440,2125.980000
1,0.021798,0.017311,-0.010543,0.044631,-0.013809,0.007838,-0.019112,0.012684,-0.002693,0.000218,0.007681,-0.314022,-0.002721,393.700000
2,0.021713,0.010882,0.012488,-0.015654,0.011082,0.004986,-0.017478,-0.003682,-0.005930,-0.000192,-0.005423,0.891683,-0.002459,748.031495
3,0.041322,0.102819,-0.029833,0.067824,0.101342,-0.023887,0.054705,-0.005680,-0.031651,-0.000720,-0.008348,-0.253258,-0.002430,787.401574
4,0.001786,0.004194,0.001742,-0.000216,-0.001976,0.006235,-0.000607,0.000235,-0.002081,0.000451,0.008637,0.532392,-0.002570,598.424000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249693,0.019544,0.061277,-0.024850,0.036257,0.073269,-0.017734,0.047925,0.000918,0.012451,-0.000118,-0.004641,-0.249980,-0.002139,1181.100000
2249694,0.020147,0.015623,0.021427,-0.000404,0.003816,0.009220,-0.025823,-0.002782,-0.011322,0.000861,0.017507,-0.148118,-0.002919,125.984252
2249695,0.063208,0.032880,0.119688,0.024788,-0.071651,-0.024792,0.045420,-0.009083,0.004295,-0.002458,-0.022181,-0.611792,-0.002125,1200.000000
2249696,0.033887,0.054742,0.086165,-0.033040,-0.014690,-0.016088,0.023324,0.032273,0.099026,-0.000348,-0.022752,-0.859387,-0.000867,2900.000000


In [7]:
train_df = dataset

In [13]:

# Split dataset into input features and output variable
X1 = train_df.iloc[:, 0:11].values
X2 = train_df.iloc[:, 11:12].values
y = train_df.iloc[:, 12].values


In [14]:
# Convert input features and output variable into numpy arrays
X1 = np.array(X1)
X2 = np.array(X2)
y = np.array(y)

# Split dataset into training and validation sets
X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(X1, X2, y, test_size=0.3, random_state=42)

In [15]:
# Normalize input features using standardization
mean = X1_train.mean(axis=0)
std = X1_train.std(axis=0)
X1_train = (X1_train - mean) / std
X1_val = (X1_val - mean) / std

In [ ]:
# Define the model
input1 = tf.keras.layers.Input(shape=(X1_train.shape[1],))
input2 = tf.keras.layers.Input(shape=(X2_train.shape[1],))

x1 = tf.keras.layers.Reshape((X1_train.shape[1], 1))(input1)
x1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x1)
x1 = tf.keras.layers.Dropout(0.2)(x1)
x1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x1)

x2 = tf.keras.layers.Dense(32)(input2)

x = tf.keras.layers.Concatenate()([x1, x2])
output = tf.keras.layers.Dense(1)(x)

model = Model(inputs=[input1, input2], outputs=output)

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

# Train the model
history = model.fit([X1_train, X2_train], y_train, epochs=10, batch_size=64, validation_data=([X1_val, X2_val], y_val))

Epoch 1/10
24607/24607 [==============================] - 562s 23ms/step - loss: 1.3539 - val_loss: 0.1746
Epoch 2/10
24607/24607 [==============================] - 807s 33ms/step - loss: 1.3539 - val_loss: 0.1746
Epoch 3/10
24607/24607 [==============================] - 1183s 48ms/step - loss: 1.3539 - val_loss: 0.1746
Epoch 4/10
24607/24607 [==============================] - 1513s 62ms/step - loss: 1.3538 - val_loss: 0.1746
Epoch 5/10
24607/24607 [==============================] - 2390s 97ms/step - loss: 1.3538 - val_loss: 0.1746
Epoch 6/10
24607/24607 [==============================] - 2726s 111ms/step - loss: 1.3538 - val_loss: 0.1746
Epoch 7/10
24607/24607 [==============================] - 3219s 131ms/step - loss: 1.3538 - val_loss: 0.1746
Epoch 8/10
17315/24607 [====================>.........] - ETA: 16:55 - loss: 0.1400

In [ ]:
testdataset = pd.read_csv('dataset/universal_test.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
# dataset =  dataset.sample(n=20000)

In [ ]:
test_df = testdataset

In [ ]:
# Extract input features from test dataset
X1_test = test_df.iloc[:, 0:11].values
X2_test = test_df.iloc[:, 11:12].values


In [ ]:
# Normalize input features using standardization
X1_test = (X1_test - mean) / std

In [ ]:
# Make predictions on test dataset
y_pred = model.predict([X1_test, X2_test])

In [ ]:
y_pred

In [25]:
y_pred1

array([[ 0.02763409,  0.02221391,  0.01022668, ..., -0.00181249,
         0.00357117,  0.00246776],
       [ 0.02797602,  0.02250296,  0.01050637, ..., -0.00158042,
         0.0037972 ,  0.00269158],
       [ 0.02802025,  0.02254035,  0.01054255, ..., -0.0015504 ,
         0.00382643,  0.00272053],
       ...,
       [ 0.02790151,  0.02243997,  0.01044542, ..., -0.001631  ,
         0.00374794,  0.0026428 ],
       [ 0.02790757,  0.02244509,  0.01045038, ..., -0.00162688,
         0.00375195,  0.00264677],
       [ 0.02800181,  0.02252476,  0.01052747, ..., -0.00156292,
         0.00381424,  0.00270846]])

In [27]:
y_pred1 = y_pred1*dataset["PRODUCT_LENGTH_NONORM"].std()  + dataset["PRODUCT_LENGTH_NONORM"].mean()

In [28]:
y_pred1

array([[41424.42172249, 34098.04169398, 17895.087537  , ...,
         1621.91900072,  8898.93893607,  7407.47394425],
       [41886.60374133, 34488.7490929 , 18273.14449826, ...,
         1935.61134148,  9204.45315439,  7710.00981119],
       [41946.38614848, 34539.2863874 , 18322.04548156, ...,
         1976.18687768,  9243.97086526,  7749.1422777 ],
       ...,
       [41785.87961718, 34403.60155589, 18190.75389293, ...,
         1867.24782887,  9137.8719141 ,  7644.07764815],
       [41794.07021915, 34410.52551368, 18197.45366481, ...,
         1872.80695708,  9143.28611296,  7649.43906587],
       [41921.45814761, 34518.2134032 , 18301.65480456, ...,
         1959.26773625,  9227.49281465,  7732.82486584]])

In [35]:
test = pd.read_csv('dataset/test.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
test = test.drop(columns = ['PRODUCT_TYPE_ID','TITLE','BULLET_POINTS','DESCRIPTION'],axis = 1)
test

,PRODUCT_ID
0,604373
1,1729783
2,1871949
3,1107571
4,624253
...,...
734731,921419
734732,2456362
734733,841529
734734,1190194


In [37]:
test["PRODUCT_LENGTH"] = y_pred1
test

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,41424.421722
1,1729783,41886.603741
2,1871949,41946.386148
3,1107571,41823.484771
4,624253,41746.889679
...,...,...
734731,921419,41903.142574
734732,2456362,41736.236768
734733,841529,41785.879617
734734,1190194,41794.070219


In [ ]:
test["PRODUCT_LENGTH"] = test["PRODUCT_LENGTH"] + 41000
test

In [39]:
test.to_csv("Namuna41000.csv", index = False)